In [1]:
import glob
import os
import numpy
import h5py
from matplotlib import pyplot as plt
import h5py

from darfix.core.dataset import Dataset
from darfix.core.dimension import POSITIONER_METADATA


In [2]:
def find_directories_with_name(root_dir, search_name):
    matching_dirs = []
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for dirname in dirnames:
            if search_name in dirname:
                matching_dirs.append(os.path.join(dirpath, dirname))
    return matching_dirs

# Function to find numbered keys in an h5 file
def find_numbered_keys(h5_file):
    numbered_keys = []
    for key in h5_file.keys():
        if key.replace('.', '').isdigit() and key.endswith('.1') :  # Check if the key is numeric
            numbered_keys.append(key)
    return numbered_keys

# Function to iterate over directories and process each .h5 file
def process_directories(directories, raw_string,metadata_string):
    file_metadata_dict = {}
    for directory in directories:
        # Find the h5 file in the current directory
        h5_files = glob.glob(os.path.join(directory, "*.h5"))
        
        if len(h5_files) == 0:
            print(f"No h5 file found in directory: {directory}")
            continue
        
        h5_file_path = h5_files[0]  # Assuming there is one .h5 file per directory
        
        # Open the h5 file and look for numbered keys
        with h5py.File(h5_file_path, 'r') as h5_file:
            numbered_keys = find_numbered_keys(h5_file)
            
            if numbered_keys:
                for key in numbered_keys:
                    raw_folder_location = f"{h5_file_path}?/{key}{raw_string}"
                    metadata_location =f"{h5_file_path}?/{key}{metadata_string}"
                    
                    # Add the file and metadata locations to the dictionary using the key as the dictionary key
                    file_metadata_dict[key] = {"raw_folder_location": raw_folder_location, "metadata_location": metadata_location}
            else:
                print(f"No numbered keys found in {h5_file_path}")

    return file_metadata_dict

In [9]:
# Example usage:
root_directory = "/dtu/3d-imaging-center/projects/2022_QIM_PMP/raw_data_extern/2024_06_beamtime/RAW_DATA/111_cells_2/"  # Replace with your root directory
search_term = "mosalayers_2x"  # Replace with the name you're looking for
directories = find_directories_with_name(root_directory, search_term)
'''for directory in directories:
    print(directory)'''
del directories[2]
for directory in directories:
    print(directory)

file_metadata_dict = process_directories(directories, '/measurement/pco_ff', '/instrument/positioners')
print(file_metadata_dict)

/dtu/3d-imaging-center/projects/2022_QIM_PMP/raw_data_extern/2024_06_beamtime/RAW_DATA/111_cells_2/111_cells_2_6-5pct_mosalayers_2x
/dtu/3d-imaging-center/projects/2022_QIM_PMP/raw_data_extern/2024_06_beamtime/RAW_DATA/111_cells_2/111_cells_2_6-1pct_mosalayers_2x_redo
/dtu/3d-imaging-center/projects/2022_QIM_PMP/raw_data_extern/2024_06_beamtime/RAW_DATA/111_cells_2/111_cells_2_6-4pct_mosalayers_2x
/dtu/3d-imaging-center/projects/2022_QIM_PMP/raw_data_extern/2024_06_beamtime/RAW_DATA/111_cells_2/111_cells_2_6-2pct_mosalayers_2x
/dtu/3d-imaging-center/projects/2022_QIM_PMP/raw_data_extern/2024_06_beamtime/RAW_DATA/111_cells_2/111_cells_2_6-6pct_mosalayers_2x
/dtu/3d-imaging-center/projects/2022_QIM_PMP/raw_data_extern/2024_06_beamtime/RAW_DATA/111_cells_2/111_cells_2_6-3pct_mosalayers_2x
/dtu/3d-imaging-center/projects/2022_QIM_PMP/raw_data_extern/2024_06_beamtime/RAW_DATA/111_cells_2/111_cells_2_6-7pct_mosalayers_2x
{'1.1': {'raw_folder_location': '/dtu/3d-imaging-center/projects/2022_Q

In [4]:
darfix_dataset_lst = []

for key, values in file_metadata_dict.items():
    darfix_dataset = Dataset(
        _dir = "/zhome/a7/7/183900/Thesis/Data",
        in_memory=True,
        first_filename=values["raw_folder_location"],
        metadata_url=values["metadata_location"],
        isH5=True
    )
    darfix_dataset_lst.append(darfix_dataset)

In [5]:
dataset = darfix_dataset_lst[0]

In [6]:
dataset.find_dimensions(POSITIONER_METADATA)

In [7]:
def plot_comparison(original, filtered, filter_name, original_name="original"):
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(15, 15), sharex=True, sharey=True)
    ax1.imshow(original, cmap=plt.cm.viridis)
    ax1.set_title(original_name)
    ax2.imshow(filtered, cmap=plt.cm.viridis)
    ax2.set_title(filter_name)

In [ ]:
#This sets the dimesions of the angles
dataset.dims.set_size(0,37)
dataset.dims.set_size(1,26)
dataset = dataset.reshape_data()

In [ ]:
#Noise removal
dataset = dataset.apply_background_subtraction(method="median")
dataset = dataset.apply_threshold_removal(bottom=10,top=64000)

In [ ]:
#Now lets compute Grain plots
moments = dataset.apply_moments()
print(moments.shape)
plot_comparison(moments[0][0], moments[1][0], "COM obpitch", "COM diffry")